# Covariates choice, preparation, cleaning and analysis

### library imports

In [1]:
# import libraries
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:

import sys
import os

# Dynamically add the 'utils' folder to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../source/')))

# Now, you can import the functions
from utils.municipality_dict import create_municipalities_dict
from utils.merge_covariates import merge_municipalities_covariates
from utils.replace_nan import replace_nan_based_on_type

## Pronovo data

In [3]:
# load cleaned pronovo data 
datapath = "../../data/cleaned_data/Swiss_solar_potential.csv"
Swiss_solar_potential_df = pd.read_csv(datapath, low_memory=False)
Swiss_solar_potential_df.set_index("mun_name", inplace=True)

## Covariate data

In [4]:
# importing population data 
population_df = pd.read_excel('../../data/raw_data/Municipal_populations_2023.xlsx', skiprows=5, skipfooter=11)
population_df.columns = ["MunicipalityNumber", "MunicipalityName", "Population"]
population_df.set_index("MunicipalityName", inplace=True)
population_df.head()

,MunicipalityNumber,Population
MunicipalityName,,
Aeugst am Albis,1,1998
Affoltern am Albis,2,12859
Bonstetten,3,5678
Hausen am Albis,4,3974
Hedingen,5,3963


In [5]:
#importing 9 urbanisation typologies 
urbanization_typo_df = pd.read_excel('../../data/raw_data/Municipalities_typologies_9.xlsx', skiprows=4, skipfooter=11)
urbanization_typo_df.columns = ["MunicipalityNumber", "MunicipalityName", "urban_typo"]
urbanization_typo_df.set_index("MunicipalityName", inplace=True)

#print(urbanization_typo_df["urban_typo"].head(10))
french_cat = [
    'Commune périurbaine de faible densité (23)',
    'Commune urbaine d’une grande agglomération (11)',
    'Commune périurbaine de moyenne densité (22)',
    'Commune périurbaine de forte densité (21)',
    "Commune urbaine d'une agglomération moyenne (12)",
    'Commune urbaine d’une petite ou hors agglomération (13)',
    'Commune rurale en situation centrale (32)',
    'Commune d’un centre rural (31)',
    'Commune rurale périphérique (33)'
]
english_cat = [
    'Low-density peri-urban municipality (23)',
    'Urban municipality in a large agglomeration (11)',
    'Medium-density peri-urban municipality (22)',
    'High-density peri-urban municipality (21)',
    'Urban municipality in a medium-sized agglomeration (12)',
    'Urban municipality in a small or non-agglomerated area (13)',
    'Centrally located rural municipality (32)',
    'Rural center municipality (31)',
    'Peripheral rural municipality (33)'
]

for i, name in enumerate(english_cat):
    urbanization_typo_df["urban_typo"] = urbanization_typo_df["urban_typo"].replace(french_cat[i], name)
    
urbanization_typo_df = urbanization_typo_df.replace('X', 0)

# visualizes df
urbanization_typo_df.tail(10)

,MunicipalityNumber,urban_typo
MunicipalityName,,
Fontenais,6790,Urban municipality in a small or non-agglomera...
Grandfontaine,6792,Peripheral rural municipality (33)
Porrentruy,6800,Urban municipality in a small or non-agglomera...
Vendlincourt,6806,Peripheral rural municipality (33)
Basse-Allaine,6807,Peripheral rural municipality (33)
Clos du Doubs,6808,Peripheral rural municipality (33)
Haute-Ajoie,6809,Peripheral rural municipality (33)
La Baroche,6810,Peripheral rural municipality (33)
Damphreux-Lugnez,6811,Peripheral rural municipality (33)


In [6]:
#imports mountainous regions
Mountain_region_df = pd.read_excel('../../data/raw_data/régions_de_montagne.xlsx', skiprows=4, skipfooter=11)
Mountain_region_df.columns = ["MunicipalityNumber", "MunicipalityName", "mountain_type"]
Mountain_region_df.set_index("MunicipalityName", inplace=True)
Mountain_region_df.tail()

,MunicipalityNumber,mountain_type
MunicipalityName,,
Lugano: inconnus,5192999,Alpes (4)
Lausanne: inconnus,5586999,Moyen-pays - Rhin supérieur (1)
La Chaux-de-Fonds: inconnus,6421999,Jura (3)
Neuchâtel: inconnus,6458999,Moyen-pays - Rhin supérieur (1)
Genève: inconnus,6621999,Moyen-pays - Rhin supérieur (1)


In [7]:
#imports mountain binary
mountain_binary_df = pd.read_excel('../../data/raw_data/mountain_binary.xlsx', skiprows=4, skipfooter=11)
mountain_binary_df.columns = ["MunicipalityNumber", "MunicipalityName", "mountain_binary"]
mountain_binary_df.set_index("MunicipalityName", inplace=True)
mountain_binary_df.tail()

,MunicipalityNumber,mountain_binary
MunicipalityName,,
Vendlincourt,6806,Communes hors des régions de montagne
Basse-Allaine,6807,Communes hors des régions de montagne
Clos du Doubs,6808,Communes des régions de montagne
Haute-Ajoie,6809,Communes hors des régions de montagne
La Baroche,6810,Communes hors des régions de montagne


In [8]:
#imports language repartition
language_df = pd.read_excel('../../data/raw_data/language_repartition.xlsx', skiprows=4, skipfooter=11)
language_df.columns = ["MunicipalityNumber", "MunicipalityName", "language"]
language_df["language"]=language_df["language"].str.replace('Région linguistique ', '', regex=False)
language_df["language"] = language_df["language"].replace("allemande", "german")
language_df["language"] = language_df["language"].replace("italienne", "italian")
language_df["language"] = language_df["language"].replace("française", "french")
language_df["language"] = language_df["language"].replace("romanche", "romansh")

language_df.set_index("MunicipalityName", inplace=True)
language_df.tail()

,MunicipalityNumber,language
MunicipalityName,,
Vendlincourt,6806,french
Basse-Allaine,6807,french
Clos du Doubs,6808,french
Haute-Ajoie,6809,french
La Baroche,6810,french


In [9]:
#imports votation repartition
votation_df = pd.read_excel('../../data/raw_data/votations_energie.xlsx', skiprows=5, skipfooter=6)
votation_df[["MunicipalityNumber", "MunicipalityName"]] = votation_df[["No commune", "Commune"]]
votation_df.drop( ["No commune","Commune"] , axis=1, inplace=True)
votation_df = votation_df[["MunicipalityName", "MunicipalityNumber", "Canton", "Oui", "Non"]]
votation_df.set_index("MunicipalityName", inplace=True)
votation_df.columns = ["MunicipalityNumber", "Canton", "Yes [nbr]", "No [nbr]"]
votation_df.tail()

,MunicipalityNumber,Canton,Yes [nbr],No [nbr]
MunicipalityName,,,,
AG-Ausland-CH,9190.0,Aargau,1847,691
TG-Ausland-CH,9200.0,Thurgau,856,313
VD-CH de l'étranger,9220.0,Vaud,4245,1186
VS-CH de l'étranger,9230.0,Valais / Wallis,1119,325
GE-CH de l'étranger,9250.0,Genève,5800,1510


In [10]:
#Municipal stats
Municipal_stats = pd.read_excel('../../data/raw_data/portrait_communes.xlsx', skiprows=5, skipfooter=16)
Municipal_stats.drop([0, 1], axis=0, inplace=True)
Municipal_stats["MunicipalityNumber"] = Municipal_stats["Code commune"]
Municipal_stats["MunicipalityName"] = Municipal_stats["Commune"]
Municipal_stats.drop("Code commune", axis=1, inplace=True)
Municipal_stats.drop("Commune", axis=1, inplace=True)
Municipal_stats = Municipal_stats[[
    'MunicipalityName', 
    "MunicipalityNumber",
    "Habitants",
    '0-19 ans', 
    '20-64 ans', 
    '65 ans ou plus',
    'Ménages privés',
    "Surfaces d'habitat et d'infrastructure en %", 
    'Emplois total', 
    'Secteur primaire',
    'Secteur secondaire', 
    'Secteur tertiaire', 
    'Nouveaux logements construits pour 1000 habitants',
    "Taux d'aide sociale",
    'PLR 2)', 
    'PDC', 
    'PS', 
    'UDC', 
    'PEV/PCS', 
    'PVL',
    'PBD', 
    'PST/Sol.', 
    'PES', 
    'Petits partis de droite'
]]

Municipal_stats.columns = [
    'MunicipalityName',
    'MunicipalityNumber',
    'inhabitants',
    '0-19 years', 
    '20-64 years', 
    '65 years or older',
    'Private households',
    'Housing and infrastructure area (%)',
    'Total employment', 
    'Primary sector',
    'Secondary sector', 
    'Tertiary sector', 
    'New housing units built per 1000 inhabitants',
    'Social assistance rate',
    'PLR',  # Swiss political party: FDP.The Liberals
    'PDC',  # Swiss political party: Christian Democratic People's Party
    'PS',   # Swiss political party: Social Democratic Party
    'UDC',  # Swiss political party: Swiss People's Party
    'PEV/PCS',  # Swiss political party: Evangelical People's Party / Christian Social Party
    'PVL',  # Swiss political party: Green Liberal Party
    'PBD',  # Swiss political party: Conservative Democratic Party
    'PST/Sol.',  # Swiss political party: Swiss Labor Party / Solidarity
    'PES',  # Swiss political party: Green Party of Switzerland
    'Small right-wing parties'
]

percentage_cols = [
    '0-19 years', 
    '20-64 years', 
    '65 years or older',
    'Housing and infrastructure area (%)',
    'Social assistance rate',
    'PLR',  # Swiss political party: FDP.The Liberals
    'PDC',  # Swiss political party: Christian Democratic People's Party
    'PS',   # Swiss political party: Social Democratic Party
    'UDC',  # Swiss political party: Swiss People's Party
    'PEV/PCS',  # Swiss political party: Evangelical People's Party / Christian Social Party
    'PVL',  # Swiss political party: Green Liberal Party
    'PBD',  # Swiss political party: Conservative Democratic Party
    'PST/Sol.',  # Swiss political party: Swiss Labor Party / Solidarity
    'PES',  # Swiss political party: Green Party of Switzerland
    'Small right-wing parties'
]

Municipal_stats = Municipal_stats.replace("X", np.nan)
Municipal_stats = Municipal_stats.replace("*", np.nan)
    
for i, col in enumerate(percentage_cols):
    if col == 'Housing and infrastructure area (%)':
        print(i)
        print("before :", Municipal_stats[col])
        Municipal_stats[col] = Municipal_stats[col]*Municipal_stats["inhabitants"]/1000
        print("after:", Municipal_stats[col])
        print("*******************************************\n")
    else:
        print(i)
        print("before :", Municipal_stats[col])
        Municipal_stats[col] = Municipal_stats[col] * Municipal_stats["inhabitants"]/100
        print("after:", Municipal_stats[col])
        print("*******************************************\n")

Municipal_stats.set_index("MunicipalityName", inplace=True)

0
before : 2       20.000000
3       19.787986
4       20.165813
5       23.636037
6       22.180752
          ...    
2170    18.425760
2171    21.728595
2172    18.289786
2173    17.788018
2174    21.366025
Name: 0-19 years, Length: 2173, dtype: float64
after: 2       1721206.6
3           392.0
4          2481.0
5          1317.0
6           832.0
          ...    
2170        103.0
2171        269.0
2172        231.0
2173        193.0
2174        244.0
Name: 0-19 years, Length: 2173, dtype: float64
*******************************************

1
before : 2       61.400000
3       62.190813
4       61.667886
5       59.834889
6       59.824047
          ...    
2170    56.887299
2171    52.988691
2172    56.611243
2173    55.483871
2174    54.028021
Name: 20-64 years, Length: 2173, dtype: float64
after: 2       5284104.262
3          1232.000
4          7587.000
5          3334.000
6          2244.000
           ...     
2170        318.000
2171        656.000
2172        715.000
217

C:\Users\godenzi\AppData\Local\Temp\ipykernel_11040\2711039545.py:80: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Municipal_stats = Municipal_stats.replace("X", np.nan)
C:\Users\godenzi\AppData\Local\Temp\ipykernel_11040\2711039545.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Municipal_stats = Municipal_stats.replace("*", np.nan)


In [11]:
Municipal_stats["inhabitants"]

MunicipalityName
Suisse                8606033.0
Aeugst am Albis          1981.0
Affoltern am Albis      12303.0
Bonstetten               5572.0
Hausen am Albis          3751.0
                        ...    
Vendlincourt              559.0
Basse-Allaine            1238.0
Clos du Doubs            1263.0
Haute-Ajoie              1085.0
La Baroche               1142.0
Name: inhabitants, Length: 2173, dtype: float64

In [12]:
# load income data
revenue_df =  pd.read_excel('../../data/raw_data/revenu_par_contribuable.xlsx', skiprows=5, skipfooter=12)
revenue_df.columns = ["MunicipalityNumber", "MunicipalityName", "Municipal_rev [CHF]", "revenue_per_capita [CHF]"]
revenue_df["Municipal_rev [CHF]"]*=1000000
revenue_df.set_index("MunicipalityName", inplace=True)

revenue_df = revenue_df.replace("X", np.nan)
revenue_df = revenue_df.replace("*", np.nan)

revenue_df.head()

C:\Users\godenzi\AppData\Local\Temp\ipykernel_11040\3076643527.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  revenue_df = revenue_df.replace("X", np.nan)


,MunicipalityNumber,Municipal_rev [CHF],revenue_per_capita [CHF]
MunicipalityName,,,
Aeugst am Albis,1,109921000.0,NaN
Affoltern am Albis,2,435719400.0,80080.757214
Bonstetten,3,251102000.0,99960.987261
Hausen am Albis,4,165486900.0,100968.212325
Hedingen,5,177331200.0,106826.024096


In [13]:
# load energy consumption data 
energy_cons_df = pd.read_csv("../../data/raw_data/energyreporter_municipality_latest.csv")
energy_cons_df = energy_cons_df[["bfs_nr", "municipality", "elec_consumption_households_mwh_per_year_per_capita", "elec_consumption_households_mwh_per_year", "elec_consumption_mwh_per_year_per_capita", "elec_consumption_mwh_per_year"]]
energy_cons_df.columns = ["MunicipalityNumber", "MunicipalityName", "elec_consumption_households_mwh_per_year_per_capita", "elec_consumption_households_mwh_per_year", "elec_consumption_mwh_per_year_per_capita", "elec_consumption_mwh_per_year"]
energy_cons_df.isna().any(axis=1)
energy_cons_df.set_index("MunicipalityName", inplace=True)
energy_cons_df.head()

,MunicipalityNumber,elec_consumption_households_mwh_per_year_per_capita,elec_consumption_households_mwh_per_year,elec_consumption_mwh_per_year_per_capita,elec_consumption_mwh_per_year
MunicipalityName,,,,,
Aeugst am Albis,1,2.9119,5811,4.4623,8905
Affoltern am Albis,2,1.6492,21056,4.1840,53420
Bonstetten,3,1.7356,9827,2.8006,15857
Hausen am Albis,4,2.3004,9128,3.9176,15545
Hedingen,5,2.1157,8335,5.3155,20941


In [14]:
# load electricity cost data 
full_elec_cost_df = pd.read_csv("../../data/raw_data/electricity_prices.csv")
operator_prices_df = full_elec_cost_df.groupby(" operatorLabel")[" total (cts./kWh)"].agg(["mean", "std"]).reset_index()
operator_prices_df.columns = ["operatorLabel", "mean_price (cts/kWh)", "std_price (cts/kWh)"]

df_mun = pd.read_csv("../../data/raw_data/electricity_municipality.csv")
print(df_mun.columns)

merged_df = pd.merge(df_mun, operator_prices_df, left_on='operator', right_on='operatorLabel')
elec_cost_df = merged_df.drop(["website", "operatorPostalCode", "operatorLabel", "operatorAddress", "operator", "canton"], axis=1)
print(len(elec_cost_df["municipalityNumber"].unique()))
elec_cost_df = elec_cost_df.drop_duplicates(subset=["municipalityNumber"])
elec_cost_df.columns = ["MunicipalityNumber", "MunicipalityName", "mean_price (cts/kWh)", "std_price (cts/kWh)"]
elec_cost_df.set_index("MunicipalityName", inplace=True)
elec_cost_df.head()

elec_cost_df.to_csv("../../data/cleaned_data/mun_cost_elec.csv")

Index(['operator', 'website', 'municipalityNumber', 'municipalityName',
       'operatorAddress', 'operatorPostalCode', 'canton'],
      dtype='object')
2079


In [15]:
# Creates a list of all covariates independent dataframes
covariates_dataframes_list = [population_df, urbanization_typo_df, Mountain_region_df, mountain_binary_df, language_df, votation_df, Municipal_stats, revenue_df, energy_cons_df, elec_cost_df]
#list of the dataframe names 
df_names_list = ["Pop_mun_df_2024","urbanization_typo_df", "Mountain_region_df", "mountain_binary_df", "language_df", "votation_df", "Municipal_stats", "revenue_df", "energy_cons_df", "elec_cost_df"]

In [16]:
# matching dataframes 
for i, cov_df in enumerate(covariates_dataframes_list):
    print(i)
    #print("Muninicipal_df :", Municipal_df.index)
    #print(f"{df_names_list[i]}:", cov_df.index)

    missing_mun = list(cov_df.index.difference(Swiss_solar_potential_df.index))
    print(f"number of elements that are in {df_names_list[i]} and not in Swiss_solar_potential_df: {len(missing_mun)}")
        
    new_mun = list(Swiss_solar_potential_df.index.difference(cov_df.index))
    print(f"elements in Swiss_solar_potential_df that aren't in {df_names_list[i]}: {len(new_mun)}")
    print("*******************************")

0
number of elements that are in Pop_mun_df_2024 and not in Swiss_solar_potential_df: 6
elements in Swiss_solar_potential_df that aren't in Pop_mun_df_2024: 1
*******************************
1
number of elements that are in urbanization_typo_df and not in Swiss_solar_potential_df: 0
elements in Swiss_solar_potential_df that aren't in urbanization_typo_df: 0
*******************************
2
number of elements that are in Mountain_region_df and not in Swiss_solar_potential_df: 1077
elements in Swiss_solar_potential_df that aren't in Mountain_region_df: 128
*******************************
3
number of elements that are in mountain_binary_df and not in Swiss_solar_potential_df: 53
elements in Swiss_solar_potential_df that aren't in mountain_binary_df: 12
*******************************
4
number of elements that are in language_df and not in Swiss_solar_potential_df: 19
elements in Swiss_solar_potential_df that aren't in language_df: 5
*******************************
5
number of elements th

### scrapping a wikipedia page that contains information on Municipalities merges throughout the years

In [17]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO  # Import StringIO

# Define the Wikipedia page URL
url = 'https://en.wikipedia.org/wiki/List_of_former_municipalities_of_Switzerland'  # Example URL

# Send a GET request to the page
response = requests.get(url)

# Parse the page content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first table on the page (You can also specify which table to scrape)
table = soup.find('table', {'class': 'wikitable'})

# Convert the table HTML to a StringIO object
table_html = str(table)
table_io = StringIO(table_html)

# Use pandas to read the table from the StringIO object
df_wiki = pd.read_html(table_io)[0]

# Display the DataFrame
df_wiki = df_wiki.sort_values(by="Year", ascending=True, inplace=False)
df_wiki = df_wiki[df_wiki.Year >= 2000]
df_wiki.to_csv("../../data/cleaned_data/mun_change.csv", index=False)

#testing
df_wiki[df_wiki.Fate.str.contains('Lugano', case=False, na=False)]

,Name,Canton[3],Fate,Resulting municipality,Year
303,Cureggia,Ticino,Incorporated into Lugano,Lugano,2004
309,Davesco-Soragno,Ticino,Incorporated into Lugano,Lugano,2004
421,Gandria,Ticino,Incorporated into Lugano,Lugano,2004
909,Pazzallo,Ticino,Incorporated into Lugano,Lugano,2004
140,Breganzona,Ticino,Incorporated into Lugano,Lugano,2004
949,Pregassona,Ticino,Incorporated into Lugano,Lugano,2004
902,Pambio-Noranco,Ticino,Incorporated into Lugano,Lugano,2004
1253,Viganello,Ticino,Incorporated into Lugano,Lugano,2004
1256,Villa Luganese,Ticino,Incorporated into Lugano,Lugano,2008
198,Carabbia,Ticino,Incorporated into Lugano,Lugano,2008


In [18]:
# Get a dictionary of merged municipalities
mun_dict = create_municipalities_dict(df_wiki)
mun_dict["Basse-Vendline"]

['Bonfol', 'Beurnevésin']

In [19]:
#getting id's from the new communes

new_muns = list(mun_dict.keys())
mask = Swiss_solar_potential_df.index.str.contains('|'.join(new_muns), case=False, na=False)
new_mun_ids_dict = Swiss_solar_potential_df.loc[mask].mun_id.to_dict()
new_mun_ids_dict["Basse-Vendline"]

6812

In [20]:
# creating a list of all dataframes 
all_cleaned_dataframes = []

In [21]:
Mountain_region_2024_df = merge_municipalities_covariates(Mountain_region_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(Mountain_region_2024_df)

In [22]:
Mountain_region_2024_df.loc["Basse-Vendline"]

MunicipalityNumber                   6812.0
mountain_type         pas de classification
Name: Basse-Vendline, dtype: object

In [23]:
Mountain_binary_2024_df = merge_municipalities_covariates(mountain_binary_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(Mountain_binary_2024_df)

In [24]:
population_2024_df = merge_municipalities_covariates(population_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(population_2024_df)

In [25]:
urbanization_typo_2024_df =  merge_municipalities_covariates(urbanization_typo_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(urbanization_typo_2024_df)

In [26]:
language_df_2024_df =  merge_municipalities_covariates(language_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(language_df_2024_df)

In [27]:
votations_2024_df =  merge_municipalities_covariates(votation_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(votations_2024_df)

Error processing non-numeric columns for target 'Glarus Süd': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Lumnezia': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Arosa': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Domleschg': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Luzein': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Jorat-Mézières': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Onsernone': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Gibloux': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Obersaxen Mundaun': single positional indexer is out-of-bounds
Error processing non-numeric columns for target 'Estavayer':

In [28]:
municipal_stats_2024_df =  merge_municipalities_covariates(Municipal_stats,mun_dict,new_mun_ids_dict, numeric_only=True)
all_cleaned_dataframes.append(municipal_stats_2024_df)

In [29]:
revenue_2024_df =  merge_municipalities_covariates(revenue_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(revenue_2024_df)

In [30]:
energy_cons_2024_df = energy_cons_df.copy()
all_cleaned_dataframes.append(energy_cons_2024_df)

In [31]:
elec_cost_2024_df = merge_municipalities_covariates(elec_cost_df,mun_dict,new_mun_ids_dict)
all_cleaned_dataframes.append(elec_cost_2024_df)

In [32]:
Mountain_region_2024_df[Mountain_region_2024_df.index.str.contains("Basse-Vendline", case=False, na=False)]

,MunicipalityNumber,mountain_type
MunicipalityName,,
Basse-Vendline,6812.0,pas de classification


In [33]:
# matching dataframes 
for i, cov_df in enumerate(all_cleaned_dataframes):
    print(i)
    #print("Muninicipal_df :", Municipal_df.index)
    #print(f"{df_names_list[i]}:", cov_df.index)

    missing_mun = list(cov_df.index.difference(Swiss_solar_potential_df.index))
    print(f"number of elements that are in {df_names_list[i]} and not in Swiss_solar_potential_df: {len(missing_mun)}")
        
    new_mun = list(Swiss_solar_potential_df.index.difference(cov_df.index))
    print(f"elements in Swiss_solar_potential_df that aren't in {df_names_list[i]}: {new_mun}")
    print(len(new_mun))
    print("*******************************")

0
number of elements that are in Pop_mun_df_2024 and not in Swiss_solar_potential_df: 283
elements in Swiss_solar_potential_df that aren't in Pop_mun_df_2024: ['Baden', 'Erlinsbach (SO)', 'Hauterive (NE)', 'Nesslau', 'Wald (BE)', 'Wangen an der Aare']
6
*******************************
1
number of elements that are in urbanization_typo_df and not in Swiss_solar_potential_df: 8
elements in Swiss_solar_potential_df that aren't in urbanization_typo_df: ['La Punt Chamues-ch', 'Nesslau']
2
*******************************
2
number of elements that are in Mountain_region_df and not in Swiss_solar_potential_df: 22
elements in Swiss_solar_potential_df that aren't in Mountain_region_df: ['Nesslau']
1
*******************************
3
number of elements that are in mountain_binary_df and not in Swiss_solar_potential_df: 1
elements in Swiss_solar_potential_df that aren't in mountain_binary_df: ['Nesslau']
1
*******************************
4
number of elements that are in language_df and not in Swis

ages > moyenner les pourcentages par rapport à la population 
ménages privés > somme
emploi > somme
logement > moyenne basique
aide sociale > pondérer par la population 
partis politiques > pondérer par population

# Merging dataframes

In [34]:
remaining_df_to_merge = [ 
    Mountain_binary_2024_df, 
    population_2024_df, 
    language_df_2024_df, 
    votations_2024_df,
    municipal_stats_2024_df, 
    revenue_2024_df,
    energy_cons_2024_df,
    elec_cost_2024_df
    
  ]

urba_final = urbanization_typo_df.reset_index("MunicipalityName")
urba_final = urba_final.set_index("MunicipalityNumber")
urba_final.head()

Mountain_region_final = Mountain_region_2024_df.reset_index("MunicipalityName")
Mountain_region_final = Mountain_region_final.set_index("MunicipalityNumber")
Mountain_region_final = Mountain_region_final.drop("MunicipalityName", axis=1)

Covariates_df = pd.merge(urba_final, Mountain_region_final, left_index=True, right_index=True, how='left')

for df in remaining_df_to_merge:
    df_final = df.reset_index("MunicipalityName")
    df_final = df_final.set_index("MunicipalityNumber")
    df_final = df_final.drop("MunicipalityName", axis=1)

    Covariates_df = pd.merge(Covariates_df, df_final, left_index=True, right_index=True, how='left')

Covariates_final_df=Covariates_df.drop("inhabitants", axis=1)
Covariates_final_df.columns

Index(['MunicipalityName', 'urban_typo', 'mountain_type', 'mountain_binary',
       'Population', 'language', 'Canton', 'Yes [nbr]', 'No [nbr]',
       '0-19 years', '20-64 years', '65 years or older', 'Private households',
       'Housing and infrastructure area (%)', 'Total employment',
       'Primary sector', 'Secondary sector', 'Tertiary sector',
       'New housing units built per 1000 inhabitants',
       'Social assistance rate', 'PLR', 'PDC', 'PS', 'UDC', 'PEV/PCS', 'PVL',
       'PBD', 'PST/Sol.', 'PES', 'Small right-wing parties',
       'Municipal_rev [CHF]', 'revenue_per_capita [CHF]',
       'elec_consumption_households_mwh_per_year_per_capita',
       'elec_consumption_households_mwh_per_year',
       'elec_consumption_mwh_per_year_per_capita',
       'elec_consumption_mwh_per_year', 'mean_price (cts/kWh)',
       'std_price (cts/kWh)'],
      dtype='object')

In [35]:
# Check if there are any duplicate indices
duplicate_indices = Covariates_final_df.index[Covariates_final_df.index.duplicated()].tolist()

# Display the rows with duplicate indices
duplicate_rows = Covariates_final_df.loc[Covariates_final_df.index.isin(duplicate_indices)]

# Show the duplicate indices and the corresponding rows
print("Duplicate indices:", duplicate_indices)
#print(duplicate_rows)
print("********************************************\n")

# Drop rows where the index is NaN
Covariates_final_df = Covariates_final_df[Covariates_final_df.index.notna()]

# Drop duplicates but keep the first occurrence
Covariates_final_df = Covariates_final_df.loc[~Covariates_final_df.index.duplicated(keep='first')]

# Verify the result
print("Data after dropping second instance of duplicated indices:")
# Check if there are any duplicate indices
duplicate_indices = Covariates_final_df.index[Covariates_final_df.index.duplicated()].tolist()

# Display the rows with duplicate indices
duplicate_rows = Covariates_final_df.loc[Covariates_final_df.index.isin(duplicate_indices)]

# Show the duplicate indices and the corresponding rows
#print("Duplicate indices:", duplicate_indices)
#print(duplicate_rows)
print("********************************************\n")

Covariates_final_df.shape

Duplicate indices: [2113.0, 2152.0, 2238.0, 2238.0, 2238.0, 3871.0, 3891.0, 3988.0, 5138.0, 5226.0, 5399.0]
********************************************

Data after dropping second instance of duplicated indices:
********************************************



(2131, 38)

In [36]:
# final tests on DF
Swiss_solar_potential_nbr_df = Swiss_solar_potential_df.reset_index()
Swiss_solar_potential_nbr_df = Swiss_solar_potential_nbr_df.set_index("mun_id")
    
missing_mun = list(Covariates_final_df.index.difference(Swiss_solar_potential_nbr_df.index))
print(f"elements that are in Covariates_final_df and not in Swiss_solar_potential_df: {missing_mun}")
    
new_mun = list(Swiss_solar_potential_nbr_df.index.difference(Covariates_final_df.index))
print(f"elements in Swiss_solar_potential_df that aren't in Covariates_final_df: {(new_mun)}")
print("*******************************")

elements that are in Covariates_final_df and not in Swiss_solar_potential_df: []
elements in Swiss_solar_potential_df that aren't in Covariates_final_df: []
*******************************


# FINAL COVARIATE DATAFRAME

In [37]:
Covariates_final_df.isna().sum()

MunicipalityName                                          0
urban_typo                                                0
mountain_type                                             9
mountain_binary                                           1
Population                                                1
language                                                  1
Canton                                                    5
Yes [nbr]                                                 5
No [nbr]                                                  5
0-19 years                                                1
20-64 years                                               1
65 years or older                                         1
Private households                                        1
Housing and infrastructure area (%)                       1
Total employment                                         88
Primary sector                                           32
Secondary sector                        

In [41]:
columns = Covariates_final_df.drop(["PLR", "PDC", "PS", "UDC", "PEV/PCS", "PVL", "PBD", "PST/Sol.", "PES", "Small right-wing parties", "Canton", "Primary sector", "Secondary sector", "Tertiary sector"], axis=1).columns
print(columns)

# Replace NaN values based on column type
updated_df = replace_nan_based_on_type(Covariates_final_df, columns)

print("\nUpdated DataFrame:")
updated_df.head()


Index(['MunicipalityName', 'urban_typo', 'mountain_type', 'mountain_binary',
       'Population', 'language', 'Yes [nbr]', 'No [nbr]', '0-19 years',
       '20-64 years', '65 years or older', 'Private households',
       'Housing and infrastructure area (%)', 'Total employment',
       'New housing units built per 1000 inhabitants',
       'Social assistance rate', 'Municipal_rev [CHF]',
       'revenue_per_capita [CHF]',
       'elec_consumption_households_mwh_per_year_per_capita',
       'elec_consumption_households_mwh_per_year',
       'elec_consumption_mwh_per_year_per_capita',
       'elec_consumption_mwh_per_year', 'mean_price (cts/kWh)',
       'std_price (cts/kWh)'],
      dtype='object')
Replaced NaN in numeric column 'Population' with mean: 3535.0291079812205
Replaced NaN in numeric column 'Yes [nbr]' with mean: 536.1274694261524
Replaced NaN in numeric column 'No [nbr]' with mean: 370.98635936030104
Replaced NaN in numeric column '0-19 years' with mean: 681.4464788731535
Re

,MunicipalityName,urban_typo,mountain_type,mountain_binary,Population,language,Canton,Yes [nbr],No [nbr],0-19 years,...,PES,Small right-wing parties,Municipal_rev [CHF],revenue_per_capita [CHF],elec_consumption_households_mwh_per_year_per_capita,elec_consumption_households_mwh_per_year,elec_consumption_mwh_per_year_per_capita,elec_consumption_mwh_per_year,mean_price (cts/kWh),std_price (cts/kWh)
MunicipalityNumber,,,,,,,,,,,,,,,,,,,,,
1.0,Aeugst am Albis,Low-density peri-urban municipality (23),Moyen-pays - Rhin supérieur (1),Communes hors des régions de montagne,1998.0,german,Zürich,444.0,335.0,392.0,...,298.149915,75.687016,109921000.0,71303.446395,2.9119,5811,4.4623,8905,26.231762,2.464933
2.0,Affoltern am Albis,Urban municipality in a large agglomeration (11),Moyen-pays - Rhin supérieur (1),Communes hors des régions de montagne,12859.0,german,Zürich,1884.0,1260.0,2481.0,...,1549.017114,191.375321,435719400.0,80080.757214,1.6492,21056,4.1840,53420,26.231762,2.464933
3.0,Bonstetten,Medium-density peri-urban municipality (22),Moyen-pays - Rhin supérieur (1),Communes hors des régions de montagne,5678.0,german,Zürich,1197.0,706.0,1317.0,...,689.174556,77.173402,251102000.0,99960.987261,1.7356,9827,2.8006,15857,26.231762,2.464933
4.0,Hausen am Albis,Medium-density peri-urban municipality (22),Moyen-pays - Rhin supérieur (1),Communes hors des régions de montagne,3974.0,german,Zürich,764.0,555.0,832.0,...,628.072197,60.419112,165486900.0,100968.212325,2.3004,9128,3.9176,15545,26.231762,2.464933
5.0,Hedingen,Urban municipality in a large agglomeration (11),Moyen-pays - Rhin supérieur (1),Communes hors des régions de montagne,3963.0,german,Zürich,809.0,580.0,817.0,...,466.535182,82.008137,177331200.0,106826.024096,2.1157,8335,5.3155,20941,26.231762,2.464933


In [39]:
updated_df.isna().sum()

MunicipalityName                                          0
urban_typo                                                0
mountain_type                                             0
mountain_binary                                           0
Population                                                0
language                                                  0
Canton                                                    5
Yes [nbr]                                                 0
No [nbr]                                                  0
0-19 years                                                0
20-64 years                                               0
65 years or older                                         0
Private households                                        0
Housing and infrastructure area (%)                       0
Total employment                                          0
Primary sector                                           32
Secondary sector                        

In [42]:
Covariates_final_df.reset_index()
Covariates_final_df.to_csv("../../data/cleaned_data/Covariates.csv", index=True, encoding='utf-8')